In [1]:
import re
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
import os,sys
import tqdm
import pickle
import time
from concurrent.futures import ThreadPoolExecutor  
import concurrent.futures

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import SGDOneClassSVM
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA
from sklearn.kernel_approximation import Nystroem
from sklearn import metrics
from sklearn.svm import OneClassSVM
import time

In [35]:
features = ['system calls frequency' ,'system calls tfidf','system calls hashing', 'system calls dependency graph', 'one hot encoding', 'dict index encoding' ]
malwares=["delay", "disorder", "freeze", "hop", "mimic", "noise", "repeat", "spoof", "normal"]

In [3]:
def reshape_matrix(matrix_list):
    new_list = [np.array(i).reshape(-1) for i in matrix_list]
    return new_list

In [4]:
def padding_onehot(onehot_list, padding):
    new_list = []
    for onehot in onehot_list:
        if len(onehot) > padding:
            onehot = np.array(onehot[0:padding])
            new_list.append(onehot)
        else:
            onehot =np.pad(onehot, [(0, padding-len(onehot)), (0, 0)], mode='constant', constant_values=0)
            new_list.append(onehot)
    new_list = reshape_matrix(new_list)
    return new_list
    

In [5]:
def padding_dictencoding(dictencoding_list, padding):
    new_list = []
    for onehot in dictencoding_list:
        if len(onehot) > padding:
            onehot = np.array(onehot[0:padding])
            new_list.append(onehot)
        else:
            onehot =np.pad(onehot, [(0, padding-len(onehot))], mode='constant', constant_values=0)
            new_list.append(onehot)
    return new_list

In [44]:
def test_models(encoded_trace_df, malware, feature, clfs):
    dfs = encoded_trace_df[encoded_trace_df.maltype==malware]
    X_test = dfs[feature].tolist()

    if feature =='system calls dependency graph':
        X_test = reshape_matrix(X_test)            
    elif feature =='one hot encoding':
        return 0,0
    elif feature =='dict index encoding':
        X_test = padding_dictencoding(X_test, 10000)       

    results = []
    preds = []

    if malware == "normal":
        y_test = np.zeros(len(X_test))
    else:
        y_test = np.ones(len(X_test))
    y_test_2 = [1 if i==0 else -1 for i in y_test]
    print(feature)
    if len(clfs) > 1:
        classifiers = clfs[1]
    else:
        classifiers = clfs[0]
    result = []
    pred = dict()    
    for name in classifiers:
        res = dict()
        clf = classifiers[name]
        t1 =time.time()
        if 'anomaly detection' in name:
            y_pred = clf.predict(X_test)
            t2 =time.time()
            score = metrics.accuracy_score(y_test_2,y_pred)
        else:
            y_pred = clf.predict(X_test)
            t2 =time.time()
            score = metrics.accuracy_score(y_test,y_pred)
        t = t2 -t1
        pred[malware +'_' + feature + '_' + name] = y_pred
        res['Model'] = malware +'_' + feature + '_' + name
        res['Accuracy'] = score
        res['Testing time'] = t
        result.append(res)
        print('Model: {}, accuracy score: {}, testing time is: {} seconds'.format( res['Model'], score, t))
    results.append(result)
    preds.append(pred)
    return results, preds       

In [26]:
# def test_models(encoded_trace_df, malware, feature, clfs, pca):
#     dfs = encoded_trace_df[encoded_trace_df.maltype==malware]
#     X_test = dfs[feature].tolist()
#     X_tv = [] 
#     if feature =='system calls dependency graph':
#         X_test = reshape_matrix(X_test)    
#         X_tv=[X_test]
#         features = [feature]
#     elif feature =='one hot encoding':
#         X_test_55000 = padding_onehot(X_test, 55000)
#         X_test_pca = pca.transform(X_test_55000)
#         X_tv = [X_test_pca]
#         features = [feature]
#     elif feature =='dict index encoding':
#         X_test_55000 = padding_dictencoding(X_test, 55000)       
#         X_test_pca = pca.transform(X_test_55000)
#         X_test_10000 = padding_dictencoding(X_test, 10000)       
#         X_tv = [X_test_pca, X_test_10000]
#         features = [feature+'-pca', feature]
#     elif feature =='system calls hashing':
#         X_tv=[X_test]
#         features = [feature]
#     else:

#         X_test_pca = pca.transform(X_test)
#         X_tv = [X_test_pca, X_test]
#         features = [feature+'-pca', feature]

#     results = []
#     preds = []
#     if malware == "normal":
#         y_test = np.zeros(len(X_test))
#     else:
#         y_test = np.ones(len(X_test))
#     y_test_2 = [1 if i==0 else -1 for i in y_test]

#     for i,classifiers in enumerate(clfs):
#         feature = features[i]
#         X_test = X_tv[i]
#         result = []
#         pred = dict()    
#         for name in classifiers:
#             res = dict()
#             clf = classifiers[name]
#             t1 =time.time()
#             if 'anomaly detection' in name:
#                 y_pred = clf.predict(X_test)
#                 t2 =time.time()
#                 score = metrics.accuracy_score(y_test_2,y_pred)
#             else:
#                 y_pred = clf.predict(X_test)
#                 t2 =time.time()
#                 score = metrics.accuracy_score(y_test,y_pred)
#             t = t2 -t1
#             pred[malware +'_' + feature + '_' + name] = y_pred
#             res['Model'] = malware +'_' + feature + '_' + name
#             res['Accuracy'] = score
#             res['Testing time'] = t
#             result.append(res)
#             print('Model: {}, accuracy score: {}, testing time is: {} seconds'.format( res['Model'], score, t))
#         results.append(result)
#         preds.append(pred)
#     return results, preds       

In [8]:
rootPath ='D:/git/IoT_Sensors_Security_Analysis/data/perf/'

In [9]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [56]:
train_device = 'pi4_2G'
test_device = 'pi3'
tw = 50

In [57]:
temppath  = rootPath + 'testset/train{}_test{}/'.format(train_device, test_device)
encoded_trace_df=pd.read_pickle(temppath+'test_{}_encoded_bow.pkl'.format(tw))

In [58]:
modelpath =  'D:/git/IoT_Sensors_Security_Analysis/results/{}/tw_{}_turn_1/'.format(train_device, tw)

In [59]:
loc=open(modelpath+'classifiers.pk','rb')
classifiers = pickle.load(loc)

loc=open(modelpath+'pcas.pk','rb')
pcas = pickle.load(loc)

In [60]:
pcas

PCA(n_components=100)

In [61]:
resultsdict = dict()
predsdict = dict()
for feature in features:
    
    clfs = classifiers[feature] 
    # testing stage
    for malware in malwares:
        results, preds = test_models(encoded_trace_df, malware, feature, clfs)
        resultsdict[malware +'_' + feature] = results
        predsdict[malware +'_' + feature] = preds

system calls frequency
Model: delay_system calls frequency_Robust covariance, accuracy score: 0.0, testing time is: 0.06301403045654297 seconds
Model: delay_system calls frequency_One-Class SVM, accuracy score: 0.0, testing time is: 0.02800607681274414 seconds
Model: delay_system calls frequency_SGD One-Class SVM, accuracy score: 1.0, testing time is: 0.0010004043579101562 seconds
Model: delay_system calls frequency_Isolation Forest, accuracy score: 0.0, testing time is: 0.058012962341308594 seconds
system calls frequency
Model: disorder_system calls frequency_Robust covariance, accuracy score: 0.0, testing time is: 0.06601476669311523 seconds
Model: disorder_system calls frequency_One-Class SVM, accuracy score: 0.0, testing time is: 0.029006481170654297 seconds
Model: disorder_system calls frequency_SGD One-Class SVM, accuracy score: 1.0, testing time is: 0.0010004043579101562 seconds
Model: disorder_system calls frequency_Isolation Forest, accuracy score: 0.0, testing time is: 0.0600

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
resultsdict['hop_one hot encoding']

0

In [ ]:
rd = []
for rs in resultsdict:
    if 'one hot' in rs:
        continue
    for r in resultsdict[rs]:
        for s in r:
            rd.append(s)


In [ ]:
rd = pd.DataFrame(rd)
md = [i.split('_') for i in rd['Model']]
md = pd.DataFrame(md)
md.columns  = ['Dataset','Features','Model']
nrd=pd.DataFrame([md['Dataset'],md['Features'],md['Model'], rd['Accuracy']]).transpose()

In [ ]:
nrd.to_csv(temppath+'results.csv',index=None)